## Measuring Tool Call Choice

This week we focused on routers to determine which functions to call or which indices to use for each user-supplied question.

This notebook shows how to benchmark function retrieval with synthetic data. It import utils.py which has some more reusable functions.

This approach mirrors how we used synthetic questions to measured document retrieval in week 1. 

## Load Products

We test tool-recall with a sample of our product inventory. We load these products below.

In [1]:
import asyncio
from typing import List
from pydantic import BaseModel
import instructor
from openai import AsyncOpenAI
import lancedb
import random

from funcs_to_call import FunctionOption
from utils import (
    calculate_precision_recall,
    FunctionList,
    QuestionWithTools,
    get_all_tool_call_evals,
    describe_tools,
)


class Product(BaseModel):
    title: str
    description: str


LANCEDB_PATH = "../week1_bootstrap_evals/lancedb"

try:
    db = lancedb.connect(LANCEDB_PATH)
    products = db.open_table("products").to_pandas()
    products = [
        Product(title=row["title"], description=row["description"])
        for _, row in products.iterrows()
    ]
except Exception as e:
    print(
        f"Error loading product data. Run the week1 course notebooks first to create the products DB"
    )
    print(f"Error: {str(e)}")

random.sample(products, 3)

/Users/dan/.pyenv/versions/3.10.0/lib/python3.10/site-packages/pydantic/_internal/_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


[Product(title='Extension Cord', description='A 50-foot extension cord with a heavy-duty jacket for outdoor use. The bright color ensures visibility.'),
 Product(title='Workbench', description='This portable workbench folds for easy storage and transport. The adjustable clamps hold your workpiece securely in place.'),
 Product(title='Circular Saw', description='A 7-1/4 inch circular saw with a powerful motor for cutting through wood and metal. The adjustable bevel and depth settings provide versatility.')]

Create a string listing all available functions to call. We will use this in our prompts.

In [2]:
tool_list = describe_tools(FunctionOption.__args__)
print(tool_list)

ShippingDateRequest: Check when a product will be shipped
ShippingCostRequest: Check the cost of shipping a product
ProductDimensionsRequest: Check the dimensions of a product
PriceHistoryRequest: Check the price history of a product (e.g. identifying historical price fluctuations)
ProductComparisonRequest: Compare two products
LogDesiredFeatureRequest: Record a user's desire for a certain product feature
ExtractDataFromImageRequest: Use our product images with multimodal llm to extract info about the product
ProductMaterialsRequest: Check what materials a product is made of


## Generate Synthetic Questions

In [3]:
async_client = instructor.from_openai(AsyncOpenAI())

def add_context_to_question(question: str, product: Product) -> str:
    return f"""Question: {question}
    
    For context, here is the product description:
    Product Description: {product.title}: {product.description}
    """


def random_tool_selection() -> List[FunctionOption]:
    num_tools = random.choice([0, 1, 2])
    return random.sample(FunctionOption.__args__, num_tools)


async def generate_synthetic_question(product: Product) -> QuestionWithTools:
    tools_to_use = random_tool_selection()
    prompt = f"""
    Create a realistic question a customer might ask a support chatbot about this product:
    {product.title}: {product.description}

    The customer knows this is a programmatic chatbot. So they will be terse and lazy (possibly skipping whole/fully formed sentences).
    """
    if tools_to_use:
        prompt += f"""The question should require using these function calls: {tool_list}
    
    Do not explicitly ask for the function. Instead, ask a question that happens to answerable by calling the function.

    For example:
    Instead of asking `how long shipping will take`, say `I need it by Friday. Can you make it?`
    Instead of asking for product dimensions, ask `Would this fit in a 3x7x4 case?`
    Instead of asking for the price history, ask `Is now a good time to buy?`

    Real questions tend to be implicit.
    Ask questions where it is hard to identify what tool(s) would help an LLM to answer the question.
    Assume that we will not make a tool call to look something up if it is already in the product description.

    Respond with the question.
    """
    else:
        prompt += f"""Respond with a question that can be answered from the product description without calling any functions:
        {tool_list}
        """

    question = await async_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": "You are creating synthetic questions for benchmarking tool retrieval in a retail chatbot.",
            },
            {"role": "user", "content": prompt},
        ],
        response_model=str,
        temperature=0.0,
    )
    tools_names = FunctionList(func_names=[tool.__name__ for tool in tools_to_use])
    question_with_context = add_context_to_question(question, product)
    return QuestionWithTools(question=question_with_context, required_tools=tools_names)


async def create_synthetic_dataset(
    products: List[Product], questions_per_product: int
) -> List[QuestionWithTools]:
    tasks = [
        generate_synthetic_question(product)
        for product in products
        for _ in range(questions_per_product)
    ]
    return await asyncio.gather(*tasks)


synthetic_questions = await create_synthetic_dataset(products, questions_per_product=2)

print(f"Generated {len(synthetic_questions)} synthetic questions. Here is a sample:")
for q in random.sample(synthetic_questions, 3):
    print(q.question)
    print(q.required_tools)
    print("---")

Generated 180 synthetic questions. Here is a sample:
Question: Can you tell me if these safety glasses have always been this expensive? Also, I need them by next week, will they arrive on time? How much will shipping cost? By the way, are they made of durable materials? And can you compare these with another pair of safety glasses? Lastly, I saw a picture of these glasses, can you get more details from it?
    
    For context, here is the product description:
    Product Description: Safety Glasses: A pair of tinted safety glasses for outdoor use. The UV protection shields your eyes from harmful rays.
    
func_names=['ProductMaterialsRequest', 'ProductDimensionsRequest']
---
Question: Will this mini hacksaw fit in small toolboxes?
    
    For context, here is the product description:
    Product Description: Hacksaw: This compact mini hacksaw is perfect for tight spaces. The high-tension blade ensures straight cuts.
    
func_names=[]
---
Question: What types of pliers are included 

## Test Whether We Call The Correct Functions

We'll have a function that's used to retrieve tools (so you can use it broadly), and then another function for evaluation

In [4]:
desired_function_calls, actual_function_calls = await get_all_tool_call_evals(
    synthetic_questions, tool_list
)
precision, recall = calculate_precision_recall(
    desired_function_calls, actual_function_calls
)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

Precision: 0.17
Recall: 0.62
